In [26]:
import duckdb
import pandas as pd
import os

# Construct the path to your database relative to the notebook location
DB_PATH = os.path.join("..", "data", "simulation.db")

# Connect to the DuckDB database
conn = duckdb.connect(DB_PATH)

In [2]:
# display orders table
df = conn.execute("SELECT * FROM order_events").fetchdf()
df.head(5)

,order_id,arrival_time,due_date,start_pick_time,end_pick_time,picker_id
0,0,2.040121,65.041412,2.040121,8.040120,Picker 1
1,1,2.683369,89.468658,2.683369,7.683369,Picker 2
2,2,5.350554,146.554489,7.683369,12.683369,Picker 2
3,3,9.805130,80.237793,9.805130,15.805130,Picker 1
4,4,10.901222,74.476891,12.683369,17.683369,Picker 2


In [27]:
cycle_time_query = """
SELECT
    AVG(end_pick_time - arrival_time) as time_in_system
FROM order_events
WHERE end_pick_time IS NOT NULL 
"""

workload_query = """
SELECT
    a.picker_id,
    c.shift_name,
    d.skill_name,
    COUNT(a.order_id) AS picked_orders
FROM 
    order_events a
    INNER JOIN pickers b ON a.picker_id = b.picker_id
    INNER JOIN shifts c ON b.shift_id = c.shift_id
    INNER JOIN skills d ON b.skill_id = d.skill_id
WHERE a.picker_id IS NOT NULL
GROUP BY a.picker_id, c.shift_name, d.skill_name
ORDER BY COUNT(a.order_id) DESC
"""


In [ ]:
conn.execute(cycle_time_query).fetchdf()

,picker_id,shift_name,skill_name,picked_orders
0,Picker 3,Night Shift,Expert,120
1,Picker 2,Day Shift,Standard,95
2,Picker 1,Day Shift,Junior,79


In [23]:
conn.execute(workload_query).fetchdf()


,picker_id,shift_name,skill_name,picked_orders
0,Picker 1,Day Shift,Junior,79
1,Picker 2,Day Shift,Standard,95
2,Picker 3,Night Shift,Expert,120


In [24]:
conn.execute("SELECT * FROM shifts").fetchdf()

,shift_id,shift_name,start_time,end_time
0,1,Day Shift,0.0,480.0
1,2,Night Shift,480.0,960.0


In [10]:
conn.execute("SELECT * FROM skills").fetchdf()

,skill_id,skill_name,speed_factor
0,1,Junior,1.2
1,2,Standard,1.0
2,3,Expert,0.8


In [ ]:
conn.close()